In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mltable
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_curve
from imblearn.over_sampling import SMOTE

In [2]:
ml_client = MLClient.from_config(credential=DefaultAzureCredential())
data_asset = ml_client.data.get("tele_customer_data", version="1")

tbl = mltable.load(f'azureml:/{data_asset.id}')

df = tbl.to_pandas_dataframe()
df.head()

Found the config file in: /config.json


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,True,False,1,False,No phone service,DSL,No,...,No,No,No,No,Month-to-month,True,Electronic check,29.85,29.85,False
1,5575-GNVDE,Male,0,False,False,34,True,No,DSL,Yes,...,Yes,No,No,No,One year,False,Mailed check,56.95,1889.50,False
2,3668-QPYBK,Male,0,False,False,2,True,No,DSL,Yes,...,No,No,No,No,Month-to-month,True,Mailed check,53.85,108.15,True
3,7795-CFOCW,Male,0,False,False,45,False,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,False,Bank transfer (automatic),42.30,1840.75,False
4,9237-HQITU,Female,0,False,False,2,True,No,Fiber optic,No,...,No,No,No,No,Month-to-month,True,Electronic check,70.70,151.65,True


In [3]:
df.columns = df.columns.str.lower().str.replace(' ','_')

categories = list(df.dtypes[df.dtypes == 'object'].index)

for c in categories:
    df[c] = df[c].str.lower().str.replace(' ','_')

In [4]:
df['totalcharges'].fillna(df['totalcharges'].mean(), inplace=True)


In [5]:
df.churn = (df.churn).astype(int)

In [6]:
from sklearn.model_selection import train_test_split
df_fulltrain, df_test = train_test_split(df, test_size= 0.2, random_state=1)


In [7]:
df_fulltrain = df_fulltrain.reset_index(drop = True)
df_test = df_test.reset_index(drop = True)

y_fulltrain = df_fulltrain.churn.values
y_test = df_test.churn.values



In [8]:
df_fulltrain.churn.value_counts(normalize= True)

0    0.730032
1    0.269968
Name: churn, dtype: float64

In [9]:
global_churn = df_fulltrain.churn.mean()
global_churn

0.26996805111821087

In [10]:
nums = ['tenure', 'monthlycharges', 'totalcharges']

cate = [ 'gender', 'seniorcitizen', 'partner', 'dependents',
        'phoneservice', 'multiplelines', 'internetservice',
       'onlinesecurity', 'onlinebackup', 'deviceprotection', 'techsupport',
       'streamingtv', 'streamingmovies', 'contract', 'paperlessbilling',
       'paymentmethod']

In [11]:
from IPython.display import display

for c in cate:
    print(c)
    df_group = df_fulltrain.groupby(c).churn.agg(['mean','count'])
    df_group['diff'] = df_group['mean'] - global_churn
    df_group['risk'] = df_group['mean'] / global_churn
    display(df_group)
    print()

gender


,mean,count,diff,risk
gender,,,,
female,0.276824,2796,0.006856,1.025396
male,0.263214,2838,-0.006755,0.974980



seniorcitizen


,mean,count,diff,risk
seniorcitizen,,,,
0,0.242270,4722,-0.027698,0.897403
1,0.413377,912,0.143409,1.531208



partner


...

,mean,count,diff,risk
contract,,,,
month-to-month,0.431701,3104,0.161733,1.599082
one_year,0.120573,1186,-0.149395,0.446621
two_year,0.028274,1344,-0.241694,0.104730



paperlessbilling


,mean,count,diff,risk
paperlessbilling,,,,
False,0.172071,2313,-0.097897,0.637375
True,0.338151,3321,0.068183,1.252560



paymentmethod


,mean,count,diff,risk
paymentmethod,,,,
bank_transfer_(automatic),0.168171,1219,-0.101797,0.622928
credit_card_(automatic),0.164339,1217,-0.105630,0.608733
electronic_check,0.455890,1893,0.185922,1.688682
mailed_check,0.193870,1305,-0.076098,0.718121


In [12]:
from sklearn.feature_extraction import DictVectorizer
#df_fulltrain = df_fulltrain.drop(columns=['churn'])
#df_test = df_test.drop(columns=['churn'])
train_dicts = df_fulltrain[cate + nums].to_dict(orient = 'records')
dv = DictVectorizer(sparse = False)
x_train = dv.fit_transform(train_dicts)
dv.get_feature_names_out()
x_train

array([[0.00000e+00, 0.00000e+00, 1.00000e+00, ..., 0.00000e+00,
        1.20000e+01, 2.58350e+02],
       [0.00000e+00, 1.00000e+00, 0.00000e+00, ..., 1.00000e+00,
        4.20000e+01, 3.16055e+03],
       [0.00000e+00, 0.00000e+00, 1.00000e+00, ..., 1.00000e+00,
        7.10000e+01, 4.68175e+03],
       ...,
       [1.00000e+00, 0.00000e+00, 0.00000e+00, ..., 0.00000e+00,
        2.80000e+01, 2.97950e+03],
       [1.00000e+00, 0.00000e+00, 0.00000e+00, ..., 0.00000e+00,
        2.00000e+00, 1.14100e+02],
       [0.00000e+00, 0.00000e+00, 1.00000e+00, ..., 1.00000e+00,
        1.60000e+01, 1.11485e+03]])

In [25]:
train_dicts = df_test[cate + nums].to_dict(orient = 'records')
y_train = dv.fit_transform(train_dicts)
y_train

array([[0.00000e+00, 1.00000e+00, 0.00000e+00, ..., 1.00000e+00,
        4.10000e+01, 3.32075e+03],
       [0.00000e+00, 0.00000e+00, 1.00000e+00, ..., 0.00000e+00,
        6.60000e+01, 6.47185e+03],
       [1.00000e+00, 0.00000e+00, 0.00000e+00, ..., 0.00000e+00,
        1.20000e+01, 5.24350e+02],
       ...,
       [0.00000e+00, 0.00000e+00, 1.00000e+00, ..., 1.00000e+00,
        7.10000e+01, 3.88865e+03],
       [1.00000e+00, 0.00000e+00, 0.00000e+00, ..., 0.00000e+00,
        6.50000e+01, 5.68845e+03],
       [1.00000e+00, 0.00000e+00, 0.00000e+00, ..., 0.00000e+00,
        1.70000e+01, 1.74350e+03]])

In [26]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

model = LogisticRegression(max_iter=200)


model.fit(x_train, y_fulltrain)

#x_test = df_test[nums+cate]
y_pred = model.predict(y_train)
model.coef_[0]
# Evaluate the model
#print(f"Accuracy: {accuracy_score(y_fulltrain, y_pred)}")
#print(classification_report(y_fulltrain, y_pred))

/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


array([ 5.94202367e-01, -8.41511825e-02, -7.02650735e-01, -7.36805956e-02,
        2.39330857e-02, -1.41714034e-01, -7.48186027e-02, -8.71251512e-02,
       -1.05474399e-01, -4.26944050e-01,  3.76058533e-01, -1.41714034e-01,
        1.10158593e-03, -2.48129835e-01,  7.21142833e-02, -1.65839987e-02,
        6.05134702e-02, -1.41714034e-01, -1.11398987e-01,  1.58887118e-01,
       -1.41714034e-01, -2.09772635e-01,  3.32867343e-01,  3.44092059e-02,
       -1.11977153e-01, -1.35980173e-01,  1.78975972e-01, -1.23618197e-01,
       -2.64713834e-01,  1.96657664e-01, -1.33738585e-01, -1.41714034e-01,
        8.28530677e-02, -8.18075676e-02, -1.41714034e-01,  3.09220506e-02,
        1.48751726e-01, -1.41714034e-01, -1.99637243e-01, -5.96511536e-02,
        3.12548437e-04])

In [27]:
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print(classification_report(y_test, y_pred))

Accuracy: 0.8112136266855926
              precision    recall  f1-score   support

           0       0.87      0.88      0.88      1061
           1       0.62      0.59      0.61       348

    accuracy                           0.81      1409
   macro avg       0.75      0.74      0.74      1409
weighted avg       0.81      0.81      0.81      1409



In [28]:
# Save the model
import joblib

# Assuming 'model' is your trained model
joblib.dump(model, 'model.pkl')

['model.pkl']

In [29]:
from azureml.core import Workspace, Experiment, Environment
from azureml.core.model import Model

# Connect to workspace
ws = Workspace.from_config()

# Create an experiment
experiment = Experiment(ws, 'churn-prediction')

# Log the experiment
run = experiment.start_logging()

In [30]:
from azureml.core import Model

# Register the model
model = Model.register(ws, model_name='churn_model', model_path='model.pkl')

Registering model churn_model


In [31]:
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import InferenceConfig

# Define the environment
inference_env = Environment.get(workspace=ws, name='AzureML-sklearn-0.24-ubuntu18.04-py37-cpu')

# Create inference configuration
inference_config = InferenceConfig(entry_script="score.py", environment=inference_env)

# Define deployment configuration
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

# Deploy the model
service = Model.deploy(workspace=ws,
                       name='churn-prediction-service',
                       models=[model],
                       inference_config=inference_config,
                       deployment_config=aci_config)

service.wait_for_deployment(show_output=True)

print(f"Scoring URI: {service.scoring_uri}")

/tmp/ipykernel_3664/3283755913.py:14: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(workspace=ws,


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2024-10-10 10:14:34+00:00 Creating Container Registry if not exists.
2024-10-10 10:14:34+00:00 Registering the environment.
2024-10-10 10:14:35+00:00 Use the existing image.
2024-10-10 10:14:38+00:00 Submitting deployment to compute.
2024-10-10 10:14:44+00:00 Checking the status of deployment churn-prediction-service.